In [ ]:
# Import library
import os
import xml.etree.ElementTree as ET
from PIL import Image
import pandas as pd
from ultralytics import YOLO
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.pt')  

# Evaluate model on test data
results = model.val(data='data.yaml', split='test')  

# Print evaluation results
print(f"mAP: {results.box.map}")
print(f"mAP@50: {results.box.map50}")
print(f"mAP@75: {results.box.map75}")
print(f"Precision: {results.box.mp}")
print(f"Recall: {results.box.mr}")

In [ ]:
# Get predictions from the model on test data
results = model.predict(source='datasets/test/images', save=False) 

# Define paths to the image and label ffor test olders
image_folder = 'datasets/test/images'  
label_folder = 'datasets/test/labels' 

# Get image names from the folder
image_names = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

# Initialize lists
y_true = []
y_pred = []

# Map labels from the folder and extract the first value before space as label
for image_name in image_names:
    label_file = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            label = f.read().strip().split()[0]  
            y_true.append(int(label)) 
    else:
        y_true.append(0)  

for result in results:
    if result.boxes is not None:
        for pred in result.boxes.cls: 
            y_pred.append(int(pred.item()))  
    else:
        y_pred.append(0)  

# Ensure y_pred has the same length as y_true by padding with default values if necessary
while len(y_pred) < len(y_true):
    y_pred.append(0)

# Create a DataFrame with image names, actual labels, and predicted labels
output_df = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': y_true,
    'Predicted Label': y_pred
})

# To save DataFrame to a CSV file
output_df.to_csv('output/custom_data_training.csv', index=False)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion  for face mask detection')
plt.savefig('output/confusion_matrix.png')
plt.show()

print("Output file and confusion matrix png saved successfully.")


# Export trained model to ONNX format
model.export(format='onnx')

# Load trained YOLOv8 model from onnx format
model = YOLO('trained_weights/exp1_yolov8n_trained.onnx') 

# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load image
    img = Image.open(image_path)
    
    # Get inference
    results = model.predict(img)
    
    # Save predicted image
    results[0].save(save_path)  

    predictions = results[0].boxes  
    print(predictions)

Image.open("output\confusion_matrix.png")

In [ ]:
# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model.predict(img)
    
    # Save predicted image
    results[0].save(save_path)  

    # To print the results
    predictions = results[0].boxes 
    print(predictions)

# Run prediction
image_path = 'datasets\predict\maksssksksss0.png'  
save_path = 'datasets\predicted\maksssksksss0.jpg' 
perform_inference(image_path, save_path)